In [1]:
# At every level, the filtering and subsampling will result in half the number of samples
# hence, half the time resolution but half the frequency band spanned, hence double the freq resolution

In [2]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn.svm import SVC, OneClassSVM
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.model_selection import StratifiedKFold, GridSearchCV, ParameterGrid
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline import Pipeline
%matplotlib inline 
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib
from mpl_toolkits.mplot3d import Axes3D #, axes3d

import pywt


In [4]:
datapath = 'tmp/features/1024_1/'
# forcesfile = datapath + 'forces_1024_1_10_20000.npz'
forcesfile = datapath + 'newprefeaturesfxyz_trans_1024_1_10_20000.npz'
# newprefeat = np.load(forcesfile, encoding = 'latin1')['f']
newprefeat = np.load(forcesfile, encoding = 'latin1')['newprefeatf']
print (newprefeat.shape,":",[fi.shape for fi in newprefeat])
# newprefeat = np.load(prefeatfile, encoding = 'latin1')['newprefeat']
# print (newprefeat.shape,":",[p.shape for p in newprefeat])


((49,), ':', [(2478, 4), (2478, 4), (2476, 4), (2464, 4), (1264, 4), (3889, 4), (2092, 4), (2019, 4), (3896, 4), (6116, 4), (6073, 4), (2447, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (2440, 4), (39998, 4), (39974, 4), (39984, 4), (39832, 4), (39926, 4), (39876, 4), (23980, 4), (20980, 4), (20480, 4), (20480, 4), (23480, 4), (23980, 4), (23980, 4), (23980, 4), (23980, 4), (23980, 4), (23980, 4), (26980, 4), (27480, 4), (27480, 4), (27480, 4)])


In [5]:
from_r = 0
# to_r = 1500
level = 15
coefs = []
for d_ind, d in enumerate(newprefeat):
    X = d[from_r:,:3]
    y = d[from_r:,3]
    coef = np.zeros_like(X)
    for n_axis in range(X.shape[1]):
        coef_tot, freq = pywt.cwt(X[:,n_axis], np.arange(1,50), 'gaus1') # coef = (lvls) x (intances)
        coef[:,n_axis] = coef_tot[level,:]
    print(coef.shape, y.shape)
    coef = np.concatenate((coef,y[:,np.newaxis]), axis = 1)
    coefs.append(coef)

((2478, 3), (2478,))
((2478, 3), (2478,))
((2476, 3), (2476,))
((2464, 3), (2464,))
((1264, 3), (1264,))
((3889, 3), (3889,))
((2092, 3), (2092,))
((2019, 3), (2019,))
((3896, 3), (3896,))
((6116, 3), (6116,))
((6073, 3), (6073,))
((2447, 3), (2447,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((2440, 3), (2440,))
((39998, 3), (39998,))
((39974, 3), (39974,))
((39984, 3), (39984,))
((39832, 3), (39832,))
((39926, 3), (39926,))
((39876, 3), (39876,))
((23980, 3), (23980,))
((20980, 3), (20980,))
((20480, 3), (20480,))
((20480, 3), (20480,))
((23480, 3), (23480,))
((23980, 3), (23980,))
((23980, 3), (23980,))
((23980, 3), (23980,))
((23980, 3), (23980,))
((23980, 3), (23980,))
((23980, 3), (23980,))
((26980, 3), (26980,)

In [9]:
%%time
# to 28 menei gia to testing, argotera na kaneis kuklikh enallagh en eidei cv metaksu twn datasets
from_d = 28
from_r = [int(np.ceil(0.1*coefs[i].shape[0])) for i in range(from_d,len(coefs))]
to_r = [int(np.ceil(0.9*coefs[i].shape[0])) for i in range(from_d,len(coefs))]
X_train = [coefs[i+from_d][from_r[i]:to_r[i],:3] for i in range(len(from_r))]
Y_train = [coefs[i+from_d][from_r[i]:to_r[i],3] for i in range(len(from_r))]

clf = [SVC(kernel = 'rbf', C = 100) for i in range(len(X_train))]
# clf = [KNeighborsClassifier(n_neighbors=7) for i in range(len(X_train))]

# asd = [print(X_train[i].shape) for i in range(len(X_train))]
nodes = [clf[i].fit(X_train[i],Y_train[i]) for i in range(len(X_train))]

CPU times: user 3min, sys: 1.18 s, total: 3min 1s
Wall time: 3min 12s


In [8]:
datapath = 'tmp/'
nodesfile = datapath + 'nodes_clf.npz'
np.savez(nodesfile, nodes = nodes)

In [6]:
# output will be calculated by a simple majority vote. 
def nodes_pred(X_test, nodes):
    Y_pred = [nodes[i].predict(X_test) for i in range(len(nodes))]
    Y_vote = np.zeros(X_test.shape[0])
    for i in range(X_test.shape[0]): # for every point in X_test 
        votes = np.zeros(2)
        for j in range(len(Y_pred)): # for every node 
            if (Y_pred[j][i]==1.):
                votes[1]+=1
            else:
                votes[0]+=1
            Y_vote[i] = np.argmax(votes)
    return Y_vote

In [33]:
wind = 1024
test_batches = np.arange(0,X_test.shape[0],wind)
x_batch = [X_test[batch:batch+wind,:] for batch in test_batches]
y_batch = [Y_test[batch:batch+wind] for batch in test_batches]

# y_batch = [[Y_pred[i][batch:batch+wind]for batch in test_batches] for i in range(len(Y_pred))]
for batch_id,batch in enumerate(test_batches):
    print(batch_id)
    Y_vot = nodes_pred(X_test = x_batch[batch_id],nodes = nodes)
    cm = confusion_matrix(y_pred=Y_vot, y_true=y_batch[batch_id])
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm)
    print(classification_report(y_pred=Y_vot, y_true = y_batch[batch_id]))


0
[[ 0.          1.        ]
 [ 0.09188661  0.90811339]]
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         1
        1.0       1.00      0.91      0.95      1023

avg / total       1.00      0.91      0.95      1024

1
[[ 1.]]
             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00      1024

avg / total       1.00      1.00      1.00      1024

2
[[ 1.]]
             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00      1024

avg / total       1.00      1.00      1.00      1024

3
[[ 1.]]
             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00      1024

avg / total       1.00      1.00      1.00      1024

4
[[ 0.  1.]
 [ 0.  1.]]
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00       116
        1.0       0.89      1.00      0.94       908

avg / total       0.79    

C:\Users\Ifoundacarrot\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ifoundacarrot\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Ifoundacarrot\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[[  0.   1.]
 [ nan  nan]]
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00      1024
        1.0       0.00      0.00      0.00         0

avg / total       0.00      0.00      0.00      1024

6
[[ 0.015625  0.984375]
 [      nan       nan]]
             precision    recall  f1-score   support

        0.0       1.00      0.02      0.03      1024
        1.0       0.00      0.00      0.00         0

avg / total       1.00      0.02      0.03      1024

7
[[ 0.22363281  0.77636719]
 [        nan         nan]]
             precision    recall  f1-score   support

        0.0       1.00      0.22      0.37      1024
        1.0       0.00      0.00      0.00         0

avg / total       1.00      0.22      0.37      1024

8
[[ 0.85449219  0.14550781]
 [        nan         nan]]
             precision    recall  f1-score   support

        0.0       1.00      0.85      0.92      1024
        1.0       0.00      0.00      0.00         0

avg 